Get all institute info from the ministry of health website

In [37]:
from pyquery import PyQuery as pq
import re
import subprocess
import json
import csv

In [28]:
search_url = "https://www.old.health.gov.il/units/pharmacy/institution/inst_out.asp?p={}"
inst_url = "https://www.old.health.gov.il/units/pharmacy/institution/inst_one1.asp?a={}"

In [39]:
def stupid_get(url):
    return subprocess.run(["curl", url], stdout=subprocess.PIPE).stdout
    

In [45]:
keys = ['בעלות', 'כתובת', 'לשכת בריאות', 'מנהל', "מס' טלפון", "מס' פקס", 'נפה', 'סוג', 'עיר/ישוב', 'קוד מוסד']
out = open('out.jsons', 'w')
page_no = 1
while True:
    print(page_no)
    url = search_url.format(page_no)
    page = stupid_get(url)
    links = pq(page).find('a')
    found = False
    for link_ in links:
        link = pq(link_)
        href = link.attr('href')
        if href.startswith('inst_one1.asp?a='):
            found = True
            inst_name = link.text()
            inst_id=int(href.split('=')[-1])
            inst_page = stupid_get(inst_url.format(inst_id))
            inst_page = pq(inst_page)
            infos = inst_page.find('td[width="50%"]')
            rec = {'name': inst_name}
            for info_ in infos:
                info = pq(info_).text()
                if ':' in info:
                    k, v = info.split(':', 1)
                    k = k.strip()
                    v = v.strip()
                    if len(k) > 0:
                        if k not in keys:
                            k, v = v, k
                        if k in keys:
                            rec[k] = v
            out.write(json.dumps(rec)+'\n')
    if not found:
        break
    page_no += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


In [49]:
import geocoder

In [55]:
out.flush()
pob = re.compile('t\.d\. [0-9,]+'.replace('t','ת').replace('d','ד'))
lines = [json.loads(line) for line in open('out.jsons')]
for line in lines:
    city = line['עיר/ישוב']
    address = line['כתובת']
    if city not in address:
        address += ' ' + city
    address = pob.sub('', address)
    ret = geocoder.google(address)
    line['lat'] = ret.lat
    line['lon'] = ret.lng
    print(address, ret)
keys = set()
for line in lines:
    keys = keys.union(line.keys())
keys = sorted(keys)
w = csv.DictWriter(open('out.csv', 'w'), keys, extrasaction='ignore')
w.writeheader()
w.writerows(lines)

רח' קפלן 14 פתח תקוה <[OK] Google - Geocode [Kaplan St 14, Petah Tikva, Israel]>
הברזל 20 תל אביב-יפו <[OK] Google - Geocode [HaBarzel St 20, Tel Aviv-Yafo, Israel]>
המגנים ,106 חיפה <[OK] Google - Geocode [Sderot HaMeginim 106, Haifa, Israel]>
הר הזיתים  ירושלים <[OK] Google - Geocode [Mount of Olives, Jerusalem]>
נצרת <[OK] Google - Geocode [Nazareth, Israel]>
הר הזיתים  ירושלים <[OK] Google - Geocode [Mount of Olives, Jerusalem]>
יאיר כץ 12 חיפה <[OK] Google - Geocode [Yair Kats St 12, Haifa, Israel]>
מגדל שבע, שד' רגר 11 באר שבע <[OK] Google - Geocode [7, שדרות יצחק רגר 11, באר שבע, 84895, Israel]>
קניון לב המפרץ חיפה <[OK] Google - Geocode [HaHistadrut Ave 55, Haifa, Israel]>
רחוב לזרוב 14 ראשון לציון <[OK] Google - Geocode [Lazarov St 14, Rishon LeTsiyon, Israel]>
הר הצופים,  ירושלים <[OK] Google - Geocode [Mt Scopus, Jerusalem]>
קרית הדסה,  ירושלים <[OK] Google - Geocode [קרית הדסה, ירושלים, 91120, Israel]>
שד' יצחק רבין עפולה <[OK] Google - Geocode [Yitshak Rabin Boulevard, Afu